# Data management project: Abritel
## Part 1: Webscrapping
**By Iman Ajdamova and Pauline Colle**

In [83]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     ----------------------- -------------- 153.6/250.0 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install xlsxwriter

     ---------------------------------------- 0.0/152.7 kB ? eta -:--:--
     ----------------------------------- -- 143.4/152.7 kB 4.3 MB/s eta 0:00:01
     -------------------------------------- 152.7/152.7 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup

import time
import os
from selenium import webdriver
from selenium.webdriver.common. by import By

import numpy as np

#vérifier à la fin qu'on a bien utiliser ça
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#Add following options before initializing the webdriver

# chromeOptions = webdriver.ChromeOptions()
# chromeOptions.add_argument("--headless")
# chromeOptions.add_argument("--remote-debugging-port=9222")
#chromeOptions.add_argument('--no-sandbox')

path="C:/Users/32498/Downloads/ECON2206-Data-Management-2023-main/chromedriver_win32"
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))
driver.get('https://www.abritel.fr/search/keywords:belgique/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true')
time.sleep(3)

#reject cookies
reject_all_button = driver.find_element(By.XPATH,"//button[contains(@class, 'osano-cm-denyAll')]")
reject_all_button.click()

time.sleep(2)


## Collecting data from the main page

In [3]:
base_url='https://www.abritel.fr/search/keywords:belgique/page:{}/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true'

num_pages = 9

all_names = []
price_night_list=[]
price_total_list=[]
rating_info=[]
types=[]
features_list=[]
premium_list = []

#loop trough all the main pages
for page_num in range(1, num_pages+1):
    link = base_url.format(page_num)
    driver.get(link)
    time.sleep(2) 
    
    #to scroll automatically
    for i in range(1, 22):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 22 * {});".format(i))
        time.sleep(1.5)
    
    #Get the names
    names = driver.find_elements(By.CSS_SELECTOR, 'h2.HitInfo__headline')
    name_list = [name.text for name in names]
    all_names.extend(name_list)
    
    #Get the prices per night
    prices = driver.find_elements(By.CSS_SELECTOR, 'span.DualPrice__secondary')
    prices_night = [price.text if price.text else None for price in prices]
    price_night_list.extend(prices_night)

    #Get the total price
    prices_total = driver.find_elements(By.CSS_SELECTOR, 'span.DualPrice__primary')
    total_list = [price_total.text if price_total.text else None for price_total in prices_total]
    price_total_list.extend(total_list)
    
    #Get the type of accomodation
    type_houses=driver.find_elements(By.CSS_SELECTOR,'div.HitInfo__content div.HitInfo__type-place-details span:first-child')
    type_list=[type_house.text for type_house in type_houses]
    types.extend(type_list)
    
    #Get if the host is a premium host or not
    elements = driver.find_elements(By.CSS_SELECTOR, 'div.HitCarousel') # trouver tous les éléments à vérifier
    for element in elements:
        premium = element.find_elements(By.CSS_SELECTOR, 'span.HitInfo__labels_item') # trouver tous les éléments "premium"
        if premium: # vérifier si l'élément "premium" existe
            premium_list.append('1') # si oui, ajouter 1
        else:
            premium_list.append('0') # sinon, ajouter 0

    #Get the rating, number of appreciation and type of announcement
    infos=driver.find_elements(By.CSS_SELECTOR,'div.HitInfo__starRating')
    for info in infos:
        rating_num = '0'
        rating_count = '0 appréciation'
        rating_status = 'rien'
    
        rating_nums = info.find_elements(By.CSS_SELECTOR, 'span.Rating__label')
        if rating_nums:
            rating_num = rating_nums[0].text

        rating_counts = info.find_elements(By.CSS_SELECTOR, 'span.Rating__count')
        if rating_counts:
            rating_count = rating_counts[0].text

        rating_3 = info.find_elements(By.CSS_SELECTOR, 'span.HitInfo__ranked')
        if rating_3:
            rating_status = rating_3[0].text
        
        rating_info.append([rating_num, rating_count, rating_status])

    for rating in rating_info:
        rating=pd.DataFrame(rating_info, columns=["Rating", "Appreciations","Type of annoucement"])                   
    
    #Get the number of people, beds and rooms
    elements = driver.find_elements(By.CSS_SELECTOR, "div.HitInfo__room-beds-details")
    for element in elements:
        span_elements = element.find_elements(By.TAG_NAME, "span")
        personnes = re.sub("[^0-9]", "", span_elements[0].text) if span_elements else '0'
        chambres = re.sub("[^0-9]", "", span_elements[1].text) if len(span_elements) > 1 else '0'
        lits = re.sub("[^0-9]", "", span_elements[2].text) if len(span_elements) > 2 else "0"
        features_list.append([personnes,chambres,lits])

features = pd.DataFrame(features_list, columns=["Number of Guests", "Number of Bedrooms", "Number of Beds"])

driver.quit()

In [31]:
data = {
    'Name': all_names,
    'Type': types,
    'hote premium':premium_list
}
# Créer le dataframe à partir du dictionnaire
df1 = pd.DataFrame(data)
df1
# Afficher le dataframe

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [32]:
data_price={
    'Price per Night': price_night_list,
    'Total Price': price_total_list,
}
dfp=pd.DataFrame(data_price)
dfp

,Price per Night,Total Price
0,122 €/nuit,1 592 €montant total
1,114 €/nuit,1 477 €montant total
2,77 €/nuit,1 005 €montant total
3,175 €/nuit,2 274 €montant total
4,100 €/nuit,1 298 €montant total
5,111 €/nuit,1 445 €montant total
6,63 €/nuit,818 €montant total
7,65 €/nuit,851 €montant total
8,67 €/nuit,870 €montant total
9,74 €/nuit,957 €montant total


In [25]:
#DataFrame for the data collected on main page
df=pd.concat([df1, dfp, rating,features], axis=1)
df.set_index('Name', inplace=True)
df.tail()

,Type,hote premium,Price per Night,Total Price,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds
Name,,,,,,,,,,
Appartement Van Hecke dans le centre-ville d'Anvers,Appartement,0,165 €/nuit,2 143 €montant total,0,(0 appréciation),Annonce professionnelle,2,1,0
Villa Oostduinkerke,Villa,0,89 €/nuit,1 156 €montant total,0,(0 appréciation),Particulier,6,3,4
"Appartement privé sise à Chaussée de Haecht, 342 1030 Schaerbeek, Bruxelle",Appartement,0,129 €/nuit,1 678 €montant total,3.5,(2 appréciations),Particulier,4,2,3
Smartflats Manneken Pis V - 1 Chambre - Centre-ville,Appartement,0,216 €/nuit,2 809 €montant total,3.3,(3 appréciations),Annonce professionnelle,2,1,1
Belle et charmante maison Perfect Location,Maison,0,NaN,NaN,3.6,(7 appréciations),Particulier,7,4,0


### Cleaning the data

In [26]:
df['Price per Night'] = df['Price per Night'].str.replace('€/nuit', '').str.replace(' ', '').astype(float)
df['Total Price'] = df['Total Price'].apply(lambda x: np.nan if pd.isna(x) or x == 'Consultez les détails pour voir le prix total' else float(x.replace(' €montant total', '').replace(' ', '').replace('€', '')))
df['Rating'] = df['Rating'].astype(float)
df['Appreciations'] = df['Appreciations'].str.extract('(\d+)').astype(float)

df['Number of Guests'] = df['Number of Guests'].astype(float)
df['Number of Bedrooms'] =df['Number of Bedrooms'].astype(float)
df['Number of Beds'] = df['Number of Beds'].astype(float)
df = df.drop(df.index[-1])#dropping the last observation because NaN

In [27]:
df.tail()

,Type,hote premium,Price per Night,Total Price,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds
Name,,,,,,,,,,
3 N appartement,Appartement,0,113.0,1472.0,0.0,0.0,Particulier,6.0,2.0,1.0
Appartement Van Hecke dans le centre-ville d'Anvers,Appartement,0,165.0,2143.0,0.0,0.0,Annonce professionnelle,2.0,1.0,0.0
Villa Oostduinkerke,Villa,0,89.0,1156.0,0.0,0.0,Particulier,6.0,3.0,4.0
"Appartement privé sise à Chaussée de Haecht, 342 1030 Schaerbeek, Bruxelle",Appartement,0,129.0,1678.0,3.5,2.0,Particulier,4.0,2.0,3.0
Smartflats Manneken Pis V - 1 Chambre - Centre-ville,Appartement,0,216.0,2809.0,3.3,3.0,Annonce professionnelle,2.0,1.0,1.0


In [28]:
#Converting in a excel file
df.to_excel('Abriteldf.xlsx',index=True)

## Localisation

In [10]:
#Opening the website
path="C:/Users/32498/Downloads/ECON2206-Data-Management-2023-main/chromedriver_win32"
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))
driver.get('https://www.abritel.fr/search/keywords:belgique/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true')
time.sleep(3)

#reject cookies
reject_all_button = driver.find_element(By.XPATH,"//button[contains(@class, 'osano-cm-denyAll')]")
reject_all_button.click()
time.sleep(2)

base_url='https://www.abritel.fr/search/keywords:belgique/page:{}/arrival:2023-09-04/departure:2023-09-17/minNightlyPrice/0/minTotalPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true'

num_pages = 9

loc_list = []
all_names = []

#Loop through the main pages
for page_num in range(1, num_pages+1):
    urls = []
    link = base_url.format(page_num)
    driver.get(link)
    time.sleep(2)
    
    #automatic scroll to display all the elements
    for i in range(1, 22):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 22 * {});".format(i))
        time.sleep(1)

    #Loop on all the accomodation to collect the URL of the specific location and adding it to a list
    annonces = driver.find_elements(By.CSS_SELECTOR, 'a.media-flex__content')
    for annonce in annonces:
        url = annonce.get_attribute('href')
        urls.append(url)
    
    #Going through all the accomodation pages 
    for url in urls:
        driver.get(url)
        time.sleep(2)
        localization = driver.find_element(By.CSS_SELECTOR,'a.u-hover-pointer.text-muted')
        localization_text = localization.text
        loc_list.append(localization_text)

        names = driver.find_elements(By.CSS_SELECTOR, 'h1.h2.margin-bottom-0x')
        name_list = [name.text for name in names]
        all_names.extend(name_list)

        driver.back()
        time.sleep(2)

driver.quit()

In [11]:
#Names of the accomodation
all_names= pd.DataFrame(all_names, columns=["Name"])
all_names

,Name
0,"95 m2 Appartement 5étage , 3 chambres, Wi-fi, ..."
1,Chalet confortable logé en forêt près de Durbuy
2,"Bel appartement pour 2 personnes avec WIFI, TV..."
3,Magnifique appartement vue sur mer
4,Près du Parlement Européen - Appartement rénov...
5,Studio Knokke
6,"Grand jardin, moyenne montagne, à la campagne,..."
7,Chalet Zutendaal
8,Ferme de Filot
9,Bertogne: Ardennes Belges - Gite récent en bor...


In [15]:
import pandas as pd
from geopy.geocoders import Nominatim

longitude = []
latitude = []

geolocator = Nominatim(user_agent="my_app")

# Browse the loc_list and retrieve the geographic coordinates for each localization
for location in loc_list:
    location = geolocator.geocode(location)
    # If the geocode function returned a value, add the longitude and latitude values to the lists
    if location is not None:
        longitude.append(location.longitude)
        latitude.append(location.latitude)
    # If the geocode function did not return a value, add NaN values to the lists
    else:
        longitude.append(float("NaN"))
        latitude.append(float("NaN"))

dfloc = pd.DataFrame({'lieu': loc_list, 'latitude': latitude, 'longitude': longitude})

print(dfloc)

                                                lieu   latitude   longitude
0                                       Koksijde-Bad  51.118739    2.634960
1                                            Barvaux  50.350609    5.493284
2                                            Bredene  51.238941    2.972449
3                                       Albertstrand  51.346426    3.275348
4                                   European Quarter  50.842350    4.383945
5                                             Knokke  51.346486    3.287631
6                                         Sankt-Vith  50.280388    6.125895
7                                          Zutendaal  50.931899    5.572555
8                                              Filot  50.426751    5.568050
9                                          Flamierge  50.060338    5.609939
10                                  European Quarter  50.842350    4.383945
11                                       Robertville  47.696274  -65.773424
12          

In [16]:
#Dataframe for the data collected through individual pages
dfloc_total=pd.concat([all_names,dfloc], axis=1)
dfloc_total.set_index('Name', inplace=True)
dfloc_total

,lieu,latitude,longitude
Name,,,
"95 m2 Appartement 5étage , 3 chambres, Wi-fi, Garage, Terrasse",Koksijde-Bad,51.118739,2.634960
Chalet confortable logé en forêt près de Durbuy,Barvaux,50.350609,5.493284
"Bel appartement pour 2 personnes avec WIFI, TV, balcon et animaux admis",Bredene,51.238941,2.972449
Magnifique appartement vue sur mer,Albertstrand,51.346426,3.275348
Près du Parlement Européen - Appartement rénové dans maison de caractère,European Quarter,50.842350,4.383945
Studio Knokke,Knokke,51.346486,3.287631
"Grand jardin, moyenne montagne, à la campagne,situation idyll. animaux acceptés",Sankt-Vith,50.280388,6.125895
Chalet Zutendaal,Zutendaal,50.931899,5.572555
Ferme de Filot,Filot,50.426751,5.568050


In [17]:
#Converting to a excel file
dfloc_total.to_excel('Abritel_loc.xlsx',index=True)

## Merging of the two main dataframe

In [20]:
# Merge the two data frames on the "Name" column
merged_df = pd.merge(df,dfloc_total, on='Name')
merged_df

,Type,hote premium,Price per Night,Total Price,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds,lieu,latitude,longitude
Name,,,,,,,,,,,,,
"95 m2 Appartement 5étage , 3 chambres, Wi-fi, Garage, Terrasse",Appartement,1,122.0,1592.0,4.5,51.0,Particulier,7.0,3.0,7.0,Koksijde-Bad,51.118739,2.634960
Chalet confortable logé en forêt près de Durbuy,Chalet,1,114.0,1477.0,4.7,59.0,Particulier,5.0,2.0,4.0,Barvaux,50.350609,5.493284
"Bel appartement pour 2 personnes avec WIFI, TV, balcon et animaux admis",Appartement,0,77.0,1005.0,5.0,1.0,Annonce professionnelle,2.0,1.0,0.0,Bredene,51.238941,2.972449
Magnifique appartement vue sur mer,Appartement,0,175.0,2274.0,0.0,0.0,Particulier,5.0,2.0,4.0,Albertstrand,51.346426,3.275348
Près du Parlement Européen - Appartement rénové dans maison de caractère,Appartement,1,100.0,1298.0,4.7,94.0,Particulier,5.0,2.0,3.0,European Quarter,50.842350,4.383945
Studio avec garage à Darwin II 0605 à étage 6,Appartement,1,111.0,1445.0,0.0,0.0,rien,5.0,1.0,0.0,Middelkerke,51.183317,2.806352
Studio avec garage à Darwin II 0605 à étage 6,Appartement,1,111.0,1445.0,0.0,0.0,rien,5.0,1.0,0.0,Middelkerke,51.183317,2.806352
"Grand jardin, moyenne montagne, à la campagne,situation idyll. animaux acceptés",Appartement,0,63.0,818.0,4.7,58.0,Particulier,4.0,1.0,3.0,Sankt-Vith,50.280388,6.125895
Chalet Zutendaal,Chalet,0,65.0,851.0,0.0,0.0,rien,4.0,2.0,0.0,Zutendaal,50.931899,5.572555


In [21]:
#Final DataFrame
merged_df.to_excel('Abritel_merged.xlsx',index=True)